# Import Libraries

In [11]:
# For text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# For topic modeling
from gensim import corpora
from gensim.models import LdaModel

# Download NLTK Resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\laila\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\laila\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\laila\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True


# Load the Data

In [12]:
documents = [
    "Rafael Nadal Joins Roger Federer in Missing U.S. Open",
    "Rafael Nadal Is Out of the Australian Open",
    "Biden Announces Virus Measures",
    "Biden's Virus Plans Meet Reality",
    "Where Biden's Virus Plan Stands"
]

# Preprocess the Data

In [13]:
stop_words = set(stopwords.words('english'))  # Create a set of English stopwords
lemmatizer = WordNetLemmatizer()  # Initialize a WordNet lemmatizer

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenize the text into words and convert to lowercase
    tokens = [token for token in tokens if token.isalnum()]  # Filter out non-alphanumeric tokens
    tokens = [token for token in tokens if token not in stop_words]  # Remove stopwords from the tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize each token
    return tokens  # Return the preprocessed tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]  # Preprocess each document in the list
preprocessed_documents

[['rafael', 'nadal', 'join', 'roger', 'federer', 'missing', 'open'],
 ['rafael', 'nadal', 'australian', 'open'],
 ['biden', 'announces', 'virus', 'measure'],
 ['biden', 'virus', 'plan', 'meet', 'reality'],
 ['biden', 'virus', 'plan', 'stand']]

# Create a document-term matrix

In [14]:
# Create a Gensim Dictionary object from the preprocessed documents
dictionary = corpora.Dictionary(preprocessed_documents) 
# Convert each preprocessed document into a bag-of-words representation using the dictionary
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]  

# Run LDA

In [15]:
#corpus: bag-of-words representation of the documents
#num_topics: number of topics to be extracted by the model
#id2word=dictionary: dictionary mapping from word IDs to words
#passes: number of passes through the corpus during training
 # Train an LDA model on the corpus with 4 topics using Gensim's LdaModel class
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15) 

# Interpret Results

In [16]:
# empty list to store dominant topic labels for each document
article_labels = []

#iterate over each processed document
for i, doc in enumerate(preprocessed_documents):
    # for each document, convert to box representation
    bow = dictionary.doc2bow(doc)
    # get list of topic probabilities
    topics = lda_model.get_document_topics(bow)
    # determine topic with highest probability
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    # appenf to the list
    article_labels.append(dominant_topic)

In [17]:
import pandas as pd

# Create DataFrame
df = pd.DataFrame({"Article": documents, "Topic": article_labels})

# Print the DataFrame
print("Table with Articles and Topic:")
print(df)
print()

Table with Articles and Topic:
                                             Article  Topic
0  Rafael Nadal Joins Roger Federer in Missing U....      1
1         Rafael Nadal Is Out of the Australian Open      1
2                     Biden Announces Virus Measures      0
3                   Biden's Virus Plans Meet Reality      0
4                    Where Biden's Virus Plan Stands      0



In [18]:
# Print the top terms for each topic
print("Top Terms for Each Topic:")
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()


Top Terms for Each Topic:
Topic 0:
- "virus" (weight: 0.167)
- "biden" (weight: 0.167)
- "plan" (weight: 0.120)
- "meet" (weight: 0.072)
- "reality" (weight: 0.072)
- "stand" (weight: 0.072)
- "announces" (weight: 0.069)
- "measure" (weight: 0.069)
- "australian" (weight: 0.024)
- "nadal" (weight: 0.024)

Topic 1:
- "open" (weight: 0.130)
- "rafael" (weight: 0.130)
- "nadal" (weight: 0.130)
- "missing" (weight: 0.078)
- "federer" (weight: 0.078)
- "roger" (weight: 0.078)
- "join" (weight: 0.078)
- "australian" (weight: 0.078)
- "measure" (weight: 0.029)
- "announces" (weight: 0.029)



Topic 0 seems to be related around politics and virus, where the weight of terms like "biden" and "virus" are particularly high, indicating their significance in this topic.

Topic 1 seems to be related to tennis, where the weight of terms like "nadal" and "rafael" are relatively high, suggesting a strong association with this topic.

In [19]:
# Print top terms for each topic
for topic_id in range(lda_model.num_topics):
    print(f"Top terms for Topic #{topic_id}:")
    top_terms = lda_model.show_topic(topic_id, topn=10)
    print([term[0] for term in top_terms])
    print()

Top terms for Topic #0:
['virus', 'biden', 'plan', 'meet', 'reality', 'stand', 'announces', 'measure', 'australian', 'nadal']

Top terms for Topic #1:
['open', 'rafael', 'nadal', 'missing', 'federer', 'roger', 'join', 'australian', 'measure', 'announces']

